In [7]:
import re
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
stopw = set(stopwords.words('english'))
from pyresparser import ResumeParser
import os
from docx import Document
# import src.notebook.skills_extraction as skills_extraction


In [16]:

# Load dataset:
jd_df = pd.read_csv(r'C:\Users\darshit joshi\Desktop\Model\src\data\jd_structured_data.csv')

# Load the extracted resume skills:
# file_path=r'C:\Users\Admin\ML_Projects\Job_Recommendation_System\Job-Recommendation-System\utilities\resumes\CV.pdf'
# skills=[]
# skills.append(' '.join(word for word in skills_extraction.skills_extractor(file_path)))
skills = ['flutter', 'application', 'development']


def ngrams(string, n=3):
    string = fix_text(string)  # fix text
    string = string.encode("ascii", errors="ignore").decode()  # remove non ascii chars
    string = string.lower()
    chars_to_remove = [")", "(", ".", "|", "[", "]", "{", "}", "'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip()  # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]



In [17]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(skills)

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
jd_test = (jd_df['Processed_JD'].values.astype('U'))

def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices



In [19]:
distances, indices = getNearestN(jd_test)
test = list(jd_test)
matches = []

for i, j in enumerate(indices):
    dist = round(distances[i][0], 2)

    temp = [dist]
    matches.append(temp)

matches = pd.DataFrame(matches, columns=['Match confidence'])
jd_df['match']=matches['Match confidence']
#jd_df.sort_values('match')
jd_df.sort_values('match', ascending=False).head(5)

,Job Title,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Competitors,Average Salary,Average Revenue,Processed_JD,match
738,Senior Data Analyst/Scientist,4.5,Demandbase,"San Francisco, CA","San Francisco, CA",350.5,2006,Company - Private,Computer Hardware & Software,Information Technology,"Engagio, Bombora, Terminus",121.5000,300.00000,The world's largest fastest-growing companies ...,1.08
566,Staff Data Engineer,3.8,Sumo Logic,"Austin, TX","Redwood City, CA",750.5,2010,Company - Private,Computer Hardware & Software,Information Technology,"Splunk, Datadog, Elastic",149.5000,24319.00076,Sumo Logic: Staff Data Engineer What do: cloud...,1.06
592,Data Scientist,4.4,Watchful,"San Francisco, CA","Apopka, FL",25.5,-1,Company - Private,Security Services,Business Services,-1,103.1539,24319.00076,"Watchful platform makes machine teaching easy,...",1.06
400,Staff Data Engineer,3.8,Sumo Logic,"Austin, TX","Redwood City, CA",750.5,2010,Company - Private,Computer Hardware & Software,Information Technology,"Splunk, Datadog, Elastic",149.5000,24319.00076,Sumo Logic: Staff Data Engineer What do: cloud...,1.06
130,Ag Data Scientist,4.6,Beck's Hybrids,"Atlanta, IN","Atlanta, IN",750.5,1937,Company - Private,Farm Support Services,Agriculture & Forestry,-1,80.5000,75.00000,We're leaders. Innovators. Perfectionists. Con...,1.06
